In [2]:
import numpy as np
from math import hypot
from imutils import face_utils
import imutils
import dlib
import cv2
from pygame import mixer
import face_recognition
import os
import glob

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        
        # Load Images 
        images_path = glob.glob(os.path.join('C:/Users/CHIRAG M/OneDrive/Desktop/Final Year Project next phase/images', "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        global fm
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                fm=1
            else:
                fm=0

        
        return fm

sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

def mid(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)



def eye_aspect_ratio(eye_landmark, face_roi_landmark):
    left_point = (face_roi_landmark.part(eye_landmark[0]).x, face_roi_landmark.part(eye_landmark[0]).y)
    right_point = (face_roi_landmark.part(eye_landmark[3]).x, face_roi_landmark.part(eye_landmark[3]).y)
    center_top = mid(face_roi_landmark.part(eye_landmark[1]), face_roi_landmark.part(eye_landmark[2]))
    center_bottom = mid(face_roi_landmark.part(eye_landmark[5]), face_roi_landmark.part(eye_landmark[4]))

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_length / ver_line_length
    return ratio

def mouth_aspect_ratio(lips_landmark, face_roi_landmark):
    left_point = (face_roi_landmark.part(lips_landmark[0]).x, face_roi_landmark.part(lips_landmark[0]).y)
    right_point = (face_roi_landmark.part(lips_landmark[2]).x, face_roi_landmark.part(lips_landmark[2]).y)
    center_top = (face_roi_landmark.part(lips_landmark[1]).x, face_roi_landmark.part(lips_landmark[1]).y)
    center_bottom = (face_roi_landmark.part(lips_landmark[3]).x, face_roi_landmark.part(lips_landmark[3]).y)

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    if hor_line_length == 0:
        return ver_line_length
    ratio = ver_line_length / hor_line_length
    return ratio

def fore_aspect_ratio(fore_landmark,face_roi_landmark):
    left_point = (face_roi_landmark.part(fore_landmark[0]).x, face_roi_landmark.part(fore_landmark[0]).y)
    right_point = (face_roi_landmark.part(fore_landmark[2]).x, face_roi_landmark.part(fore_landmark[2]).y)
    center_top = (face_roi_landmark.part(fore_landmark[1]).x, face_roi_landmark.part(fore_landmark[1]).y)
    center_bottom = (face_roi_landmark.part(fore_landmark[4]).x, face_roi_landmark.part(fore_landmark[4]).y)

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    if hor_line_length == 0:
        return ver_line_length
    ratio = ver_line_length / hor_line_length
    return ratio



detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor(".\shape_predictor_68_face_landmarks.dat")

mixer.init()
danger= mixer.Sound(r'alarm.wav')
warning=mixer.Sound(r'danger.wav')

count = 0
font = cv2.FONT_HERSHEY_TRIPLEX
cap=cv2.VideoCapture(0)
flag=0

mcount=2
fcount=0

while True:
    _, frame=cap.read()
    frame = imutils.resize(frame, width=850 , height=850)
    frame=cv2.flip(frame,180)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    global face_match
    face_match=sfr.detect_known_faces(frame)
    if(face_match):
        cv2.putText(frame,"Driver Authenticated", (380,575),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.waitKey(15)
    
    
        faces = detect(gray)
        for face_roi in faces:

            landmark_list = predict(gray, face_roi)

            left_eye_ratio = eye_aspect_ratio([36, 37, 38, 39, 40, 41], landmark_list)
            right_eye_ratio = eye_aspect_ratio([42, 43, 44, 45, 46, 47], landmark_list)
            eye_open_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        
            left_fore_ratio=fore_aspect_ratio([17,19,21,38,37,36],landmark_list)
            right_fore_ratio=fore_aspect_ratio([22,24,26,45,44,43],landmark_list)
            fore_change_ratio=(left_fore_ratio+right_fore_ratio)/2
       
            print(fore_change_ratio)
            inner_lip_ratio = mouth_aspect_ratio([60,62,64,66], landmark_list)
            outter_lip_ratio = mouth_aspect_ratio([48,51,54,57], landmark_list)
            mouth_open_ratio = (inner_lip_ratio + outter_lip_ratio) / 2;
       
            if (mouth_open_ratio > 0.380 and eye_open_ratio > 4.0) or (eye_open_ratio > 4.30) or (fore_change_ratio<0.25):
                count +=1
            else:
                count = 0
            x,y = face_roi.left(), face_roi.top()
            x1,y1 = face_roi.right(), face_roi.bottom()
            
            dflag=0
            if count>8:
                cv2.rectangle(frame, (x,y), (x1,y1), (0, 0, 255), 2)
                #cv2.putText(frame, "count:"+str(count) , (x, y-5), font, 0.5, (0, 0, 255))
                cv2.putText(frame, "****************WARNING!****************", (200,600),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                dflag=dflag+1
                warning.play()
            else:
                cv2.rectangle(frame,(x,y), (x1,y1), (0, 255, 0), 2)

        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    else:
        fcount= fcount+1
        cv2.putText(frame, "****************STRANGER!!****************", (130,600),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        cv2.waitKey(30)
        danger.play()
        cv2.imwrite('stranger.png', frame)
        from twilio.rest import Client
        client = Client("ACac0988b4c7bc8ec330a9de3c3e1a493c", "393bed38b0ae3db2a2d262eb36fcf39b")
        while(mcount>0):
            client.api.account.messages.create(to="+918970875202",from_="+18596482604",body="Stranger in car. Alert!!")
            mcount = mcount-1
        if fcount>13:
            break
cap.release()
cv2.destroyAllWindows()


2 encoding images found.
Encoding images loaded
0.48073988223482284
0.49132303901269175
0.485613853077008
0.45397525963694696
0.38497711280366614
0.45521068275557175
0.45913270914287535
0.4543177100609177
0.46715624604387174
0.4627712672650808
0.47565303980502105
0.4478875506656894
0.4515885554403841
0.4517998133257866
0.4451795058800936
0.4511652727623746
0.4682809744860441
0.46080585961071346
0.4537922944117131
0.46806767677626643
0.4818893683914415
0.47871143517650855
0.46305219206299586
0.4700478390356092
0.5121116256263605
0.45362837354162744
0.4286370159194032
0.4638443276696375
0.42457803321409326
0.4554797369493124
0.48683901514486966
0.48791349422341995
0.5323984012116723
0.5361733626908967
0.6065313273227584
0.5519125468969472
0.4817670649357638
0.48119862567709154
0.4948581352183171
0.4296647885373509
0.5104393047358254
0.435012628082642
0.49817542858790054
0.566521726617728
0.49587021865722586
0.45197358163840123
0.44478642509732347
0.45078787405491505
0.4799528085802264
0.